L'objectif est de simplement scraper et stocker sous forme de fichiers xml les différents plans de comptes

Entrée : rien

Sortie : Fichiers xml stockés dans le dossier "plan_compte"

In [249]:
import requests
from bs4 import BeautifulSoup
import os 


In [143]:
URL_MERE_PLAN_DE_COMPTE_2020 = "http://odm-budgetaire.org/composants/normes/2020/"


Stratégie naïve : 

- A partir de l'url mère, on va demander au plan de comptes d'aller chercher, pour commencer, dans l'année 2020. 
- Le script va aller chercher dans l'arborescence cette année les différentes nomenclature. 
- Dans chacune de ces nomenclatures, le script va récupérer le plan de compte xml lié
- Le script va renommer le fichier par sa nomenclature
- Le fichier renommer va être localement stocké dans un dossier "plans_de_comptes/2020/"

In [144]:
requete_mere = requests.get(URL_MERE_PLAN_DE_COMPTE_2020)
soupe_mere = BeautifulSoup(requete_mere.text, "html.parser")
soupe_mere
#On a maintenat l'arborescence, il faut récupérer uniquement les dossiers nomenclatures, pas les normes

<html>
<head><title>Index of /composants/normes/2020/</title></head>
<body>
<h1>Index of /composants/normes/2020/</h1><hr/><pre><a href="../">../</a>
<a href="M14/">M14/</a>                                               05-Mar-2020 09:43                   -
<a href="M4/">M4/</a>                                                05-Mar-2020 09:42                   -
<a href="M52/">M52/</a>                                               05-Mar-2020 09:43                   -
<a href="M57/">M57/</a>                                               05-Mar-2020 09:43                   -
<a href="M61/">M61/</a>                                               05-Mar-2020 09:43                   -
<a href="M71/">M71/</a>                                               05-Mar-2020 09:43                   -
<a href="normes-2020.xml">normes-2020.xml</a>                                    20-May-2021 15:22             1905415
<a href="normes.zip">normes.zip</a>                                         20-May-2

In [145]:
liens_dans_dossier = soupe_mere.findAll('a')
type(liens_dans_dossier)

bs4.element.ResultSet

In [146]:
liste_dossier_nom = [lien for lien in liens_dans_dossier if lien['href'].endswith('/')]
liste_dossier_nom

[<a href="../">../</a>,
 <a href="M14/">M14/</a>,
 <a href="M4/">M4/</a>,
 <a href="M52/">M52/</a>,
 <a href="M57/">M57/</a>,
 <a href="M61/">M61/</a>,
 <a href="M71/">M71/</a>]

In [147]:
tag_m14 = liste_dossier_nom[1]
tag_m14.text

'M14/'

On a maintenant accès aux liens des diff nomenclatures

In [148]:
liste_lien_nom = []
for i in liste_dossier_nom :
 test_nouvel_url = URL_MERE_PLAN_DE_COMPTE_2020 + i.text
 liste_lien_nom.append(test_nouvel_url)
liste_lien_nom

['http://odm-budgetaire.org/composants/normes/2020/../',
 'http://odm-budgetaire.org/composants/normes/2020/M14/',
 'http://odm-budgetaire.org/composants/normes/2020/M4/',
 'http://odm-budgetaire.org/composants/normes/2020/M52/',
 'http://odm-budgetaire.org/composants/normes/2020/M57/',
 'http://odm-budgetaire.org/composants/normes/2020/M61/',
 'http://odm-budgetaire.org/composants/normes/2020/M71/']

In [156]:
url_m14 = liste_lien_nom[1]
url_m14

'http://odm-budgetaire.org/composants/normes/2020/M14/'

In [151]:
requete_m14 = requests.get(liste_lien_nom[1])
requete_m14 = BeautifulSoup(requete_m14.text, 'html.parser')
type(requete_m14)

bs4.BeautifulSoup

In [163]:
lien_dans_m14 = requete_m14.findAll('a')
lien_dans_m14[1].text 
url_m14_inf3500 = url_m14 + lien_dans_m14[1].text 
url_m14_inf3500

'http://odm-budgetaire.org/composants/normes/2020/M14/M14_CCAS_INF3500/'

In [164]:
requete_inf3500 = requests.get(url_m14_inf3500)
requete_inf3500 = BeautifulSoup(requete_inf3500.text, 'html.parser')

In [251]:
lien_xml_inf3500 = requete_inf3500.find('a', string="planDeCompte.xml")
lien_xml_inf3500.text 

#url_gg = url_m14_inf3500 + lien_xml_inf3500.text 
#url_gg 

'planDeCompte.xml'

In [218]:
requete_xml_inf3500 = requests.get(url_gg)
requete_xml_inf3500.content

b'<?xml version=\'1.0\' encoding=\'ISO-8859-1\'?>\n<?xml-stylesheet type="text/xsl" href="http://odm-budgetaire.org/xsl/planCompte.xsl"?>\n<!--    Copyright\n    2010 axYus - http://www.axyus.com\n\n    This file is part of ODM print-comp.\n\n    ODM print-comp is free software; you can redistribute it and/or modify\n    it under the terms of the GNU Lesser General Public License as published by\n    the Free Software Foundation; either version 2 of the License, or\n    (at your option) any later version.\n    ODM print-comp is distributed in the hope that it will be useful,\n    but WITHOUT ANY WARRANTY; without even the implied warranty of\n    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the\n    GNU Lesser General Public License for more details.\n\n    You should have received a copy of the GNU Lesser General Public License\n    along with ODM print-comp; if not, write to the Free Software\n    Foundation, Inc., 59 Temple Place, Suite 330, Boston, MA  02111-1307  USA-

In [250]:
nom_xml = url_gg.split('normes/')[1].replace('/', '_')
nom_xml
annee_xml_test = url_gg.split('/')[5]
annee_xml_test

nomenclature_xml_test = url_gg.split('/')[6]
nomenclature_xml_test

sous_nom_text = url_gg.split('/')[7]
sous_nom_text

chemin_test = f'./{annee_xml_test}/{nomenclature_xml_test}/{sous_nom_text}/PlanDeCompte.xml'
chemin_test

chemin_complet = "./2020/M14/M14_CCAS_INF3500/PlanDeCompte.xml"

# Séparez le chemin en segments en utilisant os.path.split()
segments = chemin_complet.split("/")

# Chemin de base de la racine
chemin_base = "."

# Parcourez les segments pour créer les dossiers
for segment in segments[:-1]:  # Exclure le nom du fichier à la fin
    chemin_base = os.path.join(chemin_base, segment)
    if not os.path.exists(chemin_base):
        os.mkdir(chemin_base)

In [247]:
with open(chemin_test, 'w', encoding='utf-8') as fichier_local : 
 fichier_local.write(requete_xml_inf3500.text)

FileNotFoundError: [Errno 2] No such file or directory: './2020/M14/M14_CCAS_INF3500/PlanDeCompte.xml'

In [202]:
requete_m14
test_requete = requete_m14.find_all('a', string=lambda texte: 'M' in texte)
test_requete[1].text

for i in test_requete : 
 print(i.text)

M14_CCAS_INF3500/
M14_CCAS_SUP3500/
M14_CE/
M14_COM_500_3500/
M14_COM_INF500/
M14_COM_SUP3500/


In [195]:
def recherche_sous_nomenclature(requete) : 
 liste_exploitable = requete.findall('a', string=lambda texte : 'M' in texte)
 return liste_exploitable

In [227]:

nom_xml = url_du_xml.split('normes/')[1].replace('/', '_')
annee_xml = nom_xml.split('_')[0]
  


In [228]:
def telecharger_xml(url_du_xml) : 
 requete = requests.get(url_du_xml)
 chemin_du_xml = _creation_chemin_xml(url_du_xml)
 with open(chemin_du_xml, 'w', encoding='utf-8') as dl_local : 
  dl_local.write(requete.text)

In [237]:
url_gg

'http://odm-budgetaire.org/composants/normes/2020/M14/M14_CCAS_INF3500/planDeCompte.xml'

In [246]:
annee_xml_test = url_gg.split('/')[5]
annee_xml_test

nomenclature_xml_test = url_gg.split('/')[6]
nomenclature_xml_test

sous_nom_text = url_gg.split('/')[7]
sous_nom_text

chemin_test = f'./{annee_xml_test}/{nomenclature_xml_test}/{sous_nom_text}/PlanDeCompte.xml'
chemin_test

'./2020/M14/M14_CCAS_INF3500/PlanDeCompte.xml'